# 1. Import Libraries

In [ ]:
import torch
import evaluate
import numpy as np
import pandas as pd
import gc

from transformers import (
  T5TokenizerFast as T5Tokenizer,
  T5ForConditionalGeneration,
  Seq2SeqTrainingArguments,
  Seq2SeqTrainer,
  DataCollatorForSeq2Seq
)

from datasets import load_dataset, concatenate_datasets

# 2. Import & Preprocessing Datasets

In [ ]:
TOTAL_SAMPLES = 200
VAL_SIZE = 0.1
TEST_SIZE = 0.1

xsum = load_dataset('xsum', trust_remote_code=True, split='train')
cnn_dailymail = load_dataset('cnn_dailymail', '3.0.0', split='train')
multi_news = load_dataset('multi_news', trust_remote_code=True, split='train')

xsum = xsum.select(range(TOTAL_SAMPLES))
cnn_dailymail = cnn_dailymail.select(range(TOTAL_SAMPLES))
multi_news = multi_news.select(range(TOTAL_SAMPLES))

def process_xsum(example):
  return {
    'text': example['document'],
    'summary': example['summary'],
    'prompt': f"Summarize Harsh: {example['document']}"
  }
harsh_ds = xsum.map(process_xsum, remove_columns=xsum.column_names)
harsh_ds = harsh_ds.filter(lambda x: len(x['text'].strip()) > 0)

def process_cnn(example):
  return {
    'text': example['article'],
    'summary': example['highlights'],
    'prompt': f"Summarize Balanced: {example['article']}"
  }
balanced_ds = cnn_dailymail.map(process_cnn, remove_columns=cnn_dailymail.column_names)
balanced_ds = balanced_ds.filter(lambda x: len(x['text'].strip()) > 0)

def process_multi_news(example):
  return {
    'text': example['document'],
    'summary': example['summary'],
    'prompt': f"Summarize Detailed: {example['document']}"
  }
detailed_ds = multi_news.map(process_multi_news, remove_columns=multi_news.column_names)
detailed_ds = detailed_ds.filter(lambda x: len(x['text'].strip()) > 0)

dataset = concatenate_datasets([harsh_ds, balanced_ds, detailed_ds])

train_temp_split = dataset.train_test_split(test_size=TEST_SIZE + VAL_SIZE, shuffle=True, seed=42)
train_ds = train_temp_split['train']
temp_ds = train_temp_split['test']

val_test_split = temp_ds.train_test_split(test_size=TEST_SIZE / (TEST_SIZE + VAL_SIZE), shuffle=True, seed=42)
val_ds = val_test_split['train']
test_ds = val_test_split['test']

print(f"Train Size: {len(train_ds)}")
print(f"Validation Size: {len(val_ds)}")
print(f"Test Size: {len(test_ds)}")

Train Size: 480
Validation Size: 60
Test Size: 60


# 3. Configurations & Parameters

In [ ]:
MODEL_LIST = [
  't5-small',
  't5-base',
  'google/flan-t5-small',
  'google/flan-t5-base'
]
OUT_DIRECTORY = 'results'
MAX_INPUT_LENGTH = 512
MAX_TARGET_LENGTH = 256
BATCH_SIZE = 4
MAX_EPOCHS = 5
GRADIENT_ACCUMULATION_STEPS = 2
LEARNING_RATE = 5e-4
SEED = 42

evaluation_results = []
inference_results = []

# Text Reference: https://www.nbcnews.com/tech/tech-news/openai-disney-sora-ai-videos-rcna248617
text = '''
  The Walt Disney Co. announced Thursday that it had reached a three-year agreement with OpenAI to bring its popular characters to the company's Sora artificial intelligence video generator.
  Disney will also make a $1 billion investment in OpenAI, the owner of ChatGPT. It said it will become a “major customer” of OpenAI, using its services to develop new products and experiences, including for its Disney+ streaming service.
  “Under the agreement, Disney and OpenAI are affirming a shared commitment to the responsible use of AI that protects user safety and the rights of creators,” the companies said in a statement.
  They did not disclose the terms of the deal, and both Disney CEO Bob Iger and OpenAI CEO Sam Altman declined to reveal any details Thursday morning during a joint interview on CNBC.
  OpenAI, meanwhile, said it has committed to “implementing responsible measures to further address trust and safety, including age-appropriate policies,” but did not provide additional details about what that would entail.
  The issue of how AI chatbots engage with users under 18 is the subject of a national conversation and several lawsuits.
  Disney said characters that are part of the deal include: Mickey Mouse, Minnie Mouse, Lilo, Stitch, Ariel, Belle, Beast, Cinderella, Baymax, Simba and Mufasa, as well as characters from the worlds of “Encanto,” “Frozen,” “Inside Out,” “Moana,” “Monsters Inc.,” “Toy Story,” “Up” and “Zootopia.”
  On CNBC, Iger described the deal broadly as "kind of a way" for Disney to get into AI.
  The deal is notable in part because Disney is famously protective of its sprawling portfolio of intellectual property, from the animated shorts of the 1920s to modern superhero and fantasy franchises.
  Altman said, "We hear so much from users about how much they love Disney," adding that he expects Sora users to respond "very well" to the inclusion of Disney characters.
  The companies do not yet have a launch date yet, however, Altman said. "We'll try to get it in there as soon as we can."
  The company's statement had mentioned "early 2026" as a potential launch date.
  Iger said in a statement, “Bringing together Disney’s iconic stories and characters with OpenAI’s groundbreaking technology puts imagination and creativity directly into the hands of Disney fans in ways we’ve never seen before, giving them richer and more personal ways to connect with the Disney characters and stories they love."
  Media companies are wrestling with how to secure the value of their intellectual property while not being left behind by what many see as a transformative technology with few legal guardrails yet.
  With OpenAI, Disney would be creating a legitimate avenue through which a generative AI program could deploy its characters, rather than playing whack-a-mole with every AI company, as Disney has done with other kinds of media in the past.
'''

In [ ]:
np.random.seed(SEED)
torch.manual_seed(SEED)

rouge = evaluate.load("rouge")

# 4. Model Loading & Training

In [ ]:
def preprocess_function(examples):
  model_inputs = tokenizer(
    examples["prompt"],
    max_length=MAX_INPUT_LENGTH,
    truncation=True,
    padding="max_length",
  )
  labels = tokenizer(
    text_target=examples["summary"],
    max_length=MAX_TARGET_LENGTH,
    truncation=True,
    padding="max_length"
  )
  model_inputs["labels"] = labels["input_ids"]
  return model_inputs

In [ ]:
def compute_metrics(eval_pred):
  predictions, labels = eval_pred
  predictions = np.clip(predictions, 0, tokenizer.vocab_size - 1)

  decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

  labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
  decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

  result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
  return {k: round(v, 4) for k, v in result.items()}

In [ ]:
def generate_summary(text, style, model, tokenizer):
  model.eval()
  style_query = style.lower()
  input_text = f"Summarize {style_query}: {text}"
  input_words = len(text.split())

  inputs = tokenizer(
    input_text,
    max_length=512,
    truncation=True,
    return_tensors='pt'
  ).to(device)

  if style_query == 'harsh':
    max_len = int(input_words * 0.35)
    min_len = 5
    rep_penalty = 2.5
    beam_size = 4
  elif style_query == 'balanced':
    max_len = int(input_words * 0.50)
    min_len = 20
    rep_penalty = 1.5
    beam_size = 4
  else:
    max_len = int(input_words * 0.70)
    min_len = 50
    rep_penalty = 1.2
    beam_size = 4

  max_len = min(max_len, 250)

  with torch.no_grad():
    outputs = model.generate(
      input_ids=inputs['input_ids'],
      attention_mask=inputs['attention_mask'],
      max_length=max_len,
      min_length=min_len,
      num_beams=beam_size,
      repetition_penalty=rep_penalty,
      no_repeat_ngram_size=3,
      early_stopping=True
    )

  return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
for model_name in MODEL_LIST:
  print(f"=== Training Model: {model_name} ===")

  gc.collect()
  torch.cuda.empty_cache()

  tokenizer = T5Tokenizer.from_pretrained(model_name)
  tokenized_train = train_ds.map(preprocess_function, batched=True)
  tokenized_valid = val_ds.map(preprocess_function, batched=True)
  tokenized_test = test_ds.map(preprocess_function, batched=True)

  model = T5ForConditionalGeneration.from_pretrained(model_name)
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  model.to(device)

  training_args = Seq2SeqTrainingArguments(
    output_dir=OUT_DIRECTORY,

    num_train_epochs=MAX_EPOCHS,
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
    weight_decay=0.01,
    warmup_ratio=0.05,

    logging_steps=100,
    eval_strategy="steps",
    eval_steps=100,
    save_strategy="no",

    fp16=False,
    bf16=True,
    predict_with_generate=True,
    generation_max_length=MAX_TARGET_LENGTH,
    report_to="none",
  )

  trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_valid,
    data_collator=DataCollatorForSeq2Seq(tokenizer, model=model),
    compute_metrics=compute_metrics
  )

  trainer.train()

  val_metrics = trainer.evaluate()
  test_metrics = trainer.evaluate(eval_dataset=tokenized_test, metric_key_prefix="test")
  evaluation_results.append({
    "Model": model_name,

    "Val ROUGE-1": val_metrics.get("eval_rouge1"),
    "Val ROUGE-2": val_metrics.get("eval_rouge2"),
    "Val ROUGE-L": val_metrics.get("eval_rougeL"),
    "Val ROUGE-L Summary": val_metrics.get("eval_rougeLsum"),

    "Test ROUGE-1": test_metrics.get("test_rouge1"),
    "Test ROUGE-2": test_metrics.get("test_rouge2"),
    "Test ROUGE-L": test_metrics.get("test_rougeL"),
    "Test ROUGE-L Summary": test_metrics.get("test_rougeLsum")
  })

  styles = ["Harsh", "Balanced", "Detailed"]
  model_outputs = {"Model": model_name}

  for style in styles:
    summary = generate_summary(text, style, model, tokenizer)
    model_outputs[style] = summary

  inference_results.append(model_outputs)

  del model
  del trainer
  del tokenizer
  del tokenized_train
  del tokenized_valid
  del tokenized_test

=== Training Model: t5-small ===


Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
100,2.720300,1.245964,0.174800,0.052200,0.111700,0.112000
200,1.256700,1.236250,0.258600,0.069500,0.165100,0.165300
300,1.179700,1.235483,0.258300,0.067200,0.165400,0.164900


=== Training Model: t5-base ===


Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
100,2.096500,1.074870,0.282500,0.071700,0.185500,0.185100
200,0.862600,1.117765,0.289300,0.091800,0.193700,0.193700
300,0.695500,1.137300,0.307800,0.094000,0.205400,0.205400


=== Training Model: google/flan-t5-small ===


Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
100,5.176900,1.115493,0.253300,0.069300,0.183000,0.183000
200,1.114500,1.122908,0.301100,0.080700,0.202100,0.203000
300,1.006800,1.131390,0.292400,0.080000,0.198200,0.199100


=== Training Model: google/flan-t5-base ===


Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
100,4.015600,0.981331,0.310300,0.093400,0.210400,0.210800
200,0.805700,1.033992,0.322900,0.098600,0.211400,0.211200
300,0.645600,1.051101,0.324300,0.091800,0.207500,0.207700


# 6. Model Evaluation Comparison

In [ ]:
rouge_df = pd.DataFrame(evaluation_results)
display(rouge_df)

,Model,Val ROUGE-1,Val ROUGE-2,Val ROUGE-L,Val ROUGE-L Summary,Test ROUGE-1,Test ROUGE-2,Test ROUGE-L,Test ROUGE-L Summary
0,t5-small,0.2583,0.0672,0.1654,0.1649,0.2885,0.0795,0.1862,0.1864
1,t5-base,0.3078,0.0940,0.2054,0.2054,0.3346,0.1005,0.2130,0.2126
2,google/flan-t5-small,0.2924,0.0800,0.1982,0.1991,0.3049,0.0864,0.2031,0.2028
3,google/flan-t5-base,0.3243,0.0918,0.2075,0.2077,0.3484,0.1117,0.2268,0.2272


In [ ]:
style_df = pd.DataFrame(inference_results)
display(style_df.set_index("Model"))

,Harsh,Balanced,Detailed
Model,,,
t5-small,The Walt Disney Co. has reached a three-year d...,The Walt Disney Co. has reached a three-year d...,– The Walt Disney Co. has reached a three-year...
t5-base,Disney and OpenAI have signed a three-year dea...,Disney and OpenAI have signed a three-year dea...,Disney and OpenAI have signed a three-year dea...
google/flan-t5-small,The Walt Disney Co. has reached a three-year a...,The Walt Disney Co. has reached a three-year a...,The Walt Disney Co. has reached a three-year a...
google/flan-t5-base,Disney has reached a three-year agreement with...,Disney has reached a three-year agreement with...,– Disney has reached a three-year deal with Op...
